In [1]:
from common import *

Using TensorFlow backend.


# Load data

In [2]:
model = gensim.models.Word2Vec.load(join(DATA_FOLDER, 'vectors/w2v_model_300_w10'))
wv = model.wv

2017-07-31 22:55:47,495 : INFO : loading Word2Vec object from ../data/vectors/w2v_model_300_w10
2017-07-31 22:55:49,557 : INFO : loading wv recursively from ../data/vectors/w2v_model_300_w10.wv.* with mmap=None
2017-07-31 22:55:49,558 : INFO : loading syn0 from ../data/vectors/w2v_model_300_w10.wv.syn0.npy with mmap=None
2017-07-31 22:55:50,494 : INFO : setting ignored attribute syn0norm to None
2017-07-31 22:55:50,495 : INFO : loading syn1neg from ../data/vectors/w2v_model_300_w10.syn1neg.npy with mmap=None
2017-07-31 22:55:51,411 : INFO : setting ignored attribute cum_table to None
2017-07-31 22:55:51,414 : INFO : loaded ../data/vectors/w2v_model_300_w10


In [3]:
dictionary = corpora.Dictionary.load(join(DATA_FOLDER, 'pure.dict'))
tfidf = models.TfidfModel.load(join(DATA_FOLDER, 'tfidf_pure.model'))
# !!!!!
tfidf.normalize= True

2017-07-31 22:55:53,242 : INFO : loading Dictionary object from ../data/pure.dict
2017-07-31 22:55:54,615 : INFO : loaded ../data/pure.dict
2017-07-31 22:55:54,616 : INFO : loading TfidfModel object from ../data/tfidf_pure.model
2017-07-31 22:55:55,042 : INFO : loaded ../data/tfidf_pure.model


In [4]:
all_docs = get_all_docs(DATA_FOLDER)
val_docs = sorted(glob(join(DATA_FOLDER, 'validate/*.txt')))

# Main funcs

In [6]:
def sim_matrix(word_set1, word_set2, wv):
    l1 = list(word_set1)
    l2 = list(word_set2)
    wv1 = wv[l1]
    wv2 = wv[l2]
    
    arr = cosine_similarity(wv1, wv2)
    
    smat = pd.DataFrame(arr, index=l1, columns=l2)
    return smat
    

In [7]:
def wmd(smat, tfidf_weights):
    maxs = np.amax(smat, axis=0)
    return np.dot(maxs, tfidf_weights)

In [8]:
def get_test_tfidf_weights(val_docs, wv):
    tweights = {}
    for fname in tqdm(val_docs):
        with open(fname, 'r') as f:
            doc_text = f.read()
        tokenized_filtered = [w for w in tokenize(doc_text) if w in wv]
        doc_bow = dictionary.doc2bow(tokenized_filtered)
        sorted_tfidf = sorted(tfidf[doc_bow], key=itemgetter(1), reverse=True)
        sorted_tfidf = pd.DataFrame([(dictionary[k],v) for k,v in sorted_tfidf],
                                   columns=['word', 'score'])
        
        name = path.splitext(basename(fname))[0]
        tweights[name] = sorted_tfidf
            
    return tweights

In [9]:
tweights = get_test_tfidf_weights(val_docs, wv)
val_words = set(pd.concat([v.word for v in tweights.values()]))
ziped_files = sorted(glob(DATA_FOLDER + '/documents/*.gz'), key=natural_keys)

100%|██████████| 130/130 [00:00<00:00, 197.61it/s]


In [10]:
# tweights['_01_98_2010120931A10010101RU']

In [11]:
def doc_queries_distances(d_smat, tweights):
    dists = []
    for k, query in tweights.items():
        if len(query) == 0:
            dists.append(None)
            continue
        
        qsmat = d_smat.loc[:, query.word]
        dist = wmd(qsmat, query.score)
        dists.append(dist)
    return dists


def _calc(docs):        
    doc_dists = []
    words = set([w for doc in docs for sent in doc for w in sent if w in wv])
    smat = sim_matrix(words, val_words, wv)
    for doc in tqdm(docs):
        doc_words = set([w for sent in doc for w in sent])
        d_smat = smat.loc[doc_words, :]
        dists = doc_queries_distances(d_smat, tweights)

        doc_dists.append(dists)
    return doc_dists


def calc_dists(fnames, wv):
    doc_dists = []
    for fn in fnames:
        print(fn)
        ! echo {fn} >> ../data/out
        with GzipFile(fn, 'r') as myzip:
            text = myzip.read()
        docs = json.loads(text)
        
        parallelizer = Parallel(n_jobs=cpu_count)

        # this iterator returns the functions to execute for each task
        tasks_iterator = ( delayed(_calc)(docs_block) for 
                          docs_block in grouper(len(docs)//cpu_count, docs) )  
        result = parallelizer( tasks_iterator )        
        doc_dists += result
            
    return np.array(sum(doc_dists, []))

In [128]:
val_dists = calc_dists(ziped_files[:1], wv)

../data//documents/0.json.gz


100%|██████████| 298/298 [01:49<00:00,  4.37it/s]


In [132]:
np.save(join(DATA_FOLDER, 'val_dists.npy'), val_dists)

In [136]:
val_dists = np.load(join(DATA_FOLDER, 'val_dists.npy'))

In [137]:
sorted_args = val_dists.argsort(axis=0)[-20000000:][::-1]

In [138]:
sorted_args

array([[  20,    0,  857, ..., 1121,  778,  759],
       [ 563,  474,  563, ..., 1122,  792,  876],
       [ 857,   20,  482, ...,  792,  625, 1012],
       ..., 
       [ 327,  327, 1028, ...,  915, 1028,  313],
       [1044,  583,  327, ..., 1028, 1175, 1175],
       [1028, 1044, 1044, ..., 1192, 1192, 1028]])

In [142]:
send_email(notebook_url, body='Finished')

Successfully sent email


TODO:
* normalize = False in tfidf
* first 2-3 sents of query

# Check

In [27]:
! cat {all_docs[20]}

способ изготовление портландцемент и способ изготовление бетонный и железобетонный изделие на основа изготовлять портландцемент  . 
 изобретение относиться к технология строительный материал и мочь быть использовать для производство цемент и изделие из бетон .
 сущность изобретение заключаться в том , что в способ изготовление цемент , включать получение алитовый портландцементный клинкер обжиг исходный сырьевой смесь до спекание во вращать печь с введение добавка портландцемент в количество 0,05 - 5% масса клинкер с последующий охлаждение клинкер и помол о совместно с гипсовый компонент , в цементный сырьевой смесь дополнительно вводить цементный пыль , улавливать из отходить печной газ , при первоначальный разность λ значение коэффициент насыщение сырьевой смесь и пыль 0,2 - 0,4 , добавка портландцемент сначала вводить в количество 0,05 мас . % , затем увеличивать ее содержание в 1,5 - 5 раз до получение l в предел 0,1 - 0,15 , далее , не менять подача добавка портландцемент , увелич

In [17]:
! cat {val_docs[0]}

﻿способ экономия электроэнергия .
 изобретение относиться к проблема экономия электроэнергия , как один из весомый в программа борьба с глобальный потепление климат , и эффективный мочь быть применять , как это ни показаться неожиданный , в область кулинарный хозяйство .
 проблема с электроэнергия возникнуть сразу же после изобретение в 1879 г . гениальный томас альва эдисон электрический лампа накаливание .
 при ее запиток от использовать тогда источник постоянный ток , недопустимый большой быть потеря в линия передача на большой расстояние .
 проблема быть устранять в 1887 г . один из великий физик современность никола теслый , свой изобретение генератор переменный ток и высоковольтный трансформатор .
 дальнейший нововведение , как и этот , подстраиваться , в основной , к электрический характеристика объект запиток , как правило не учитывать они физический особенность .
 в кулинария экономия затрачивать энергия не являться главенствующий фактор , но неизменный оставаться только стрем

In [21]:
tweights['_01_98_2010120931A10010101RU']

,word,score
0,экономия,0.419251
1,кулинария,0.253144
2,электроэнергия,0.234901
3,томить,0.218281
4,блюдо,0.196800
5,старинный,0.183828
6,запиток,0.150455
7,кухня,0.134959
8,настаивать,0.132950
9,бытовой,0.130700


In [26]:
all_docs[1028]

'../data/FIPS/Inventions_patents_txt_output/06/a5/0002072734C119970127RU.txt'

In [24]:
doc = docs[563]
words = set([w for s in doc for w in s if w in wv])
smat = sim_matrix(words, val_words, wv)

query = tweights['_01_98_2010120931A10010101RU']
qsmat = smat.loc[:, query.word]
dist = wmd(qsmat, query.score)
dist


6.4343087036007391


# First n

In [19]:
# fn = join(DATA_FOLDER, 'first2.json.gz')

# with GzipFile(fn, 'r') as myzip:
#     text = myzip.read()
# docs = json.loads(text)
# del text

In [26]:
fn = join(DATA_FOLDER, 'first3_val.json.gz')

with GzipFile(fn, 'r') as myzip:
    text = myzip.read()
val_documents = json.loads(text)
del text

In [27]:
len(val_documents)

130

In [47]:
tweights = {}
for fname, doc in tqdm(zip(val_docs, val_documents)):
    tokenized_filtered = [w for w in doc if w in wv]
    doc_bow = dictionary.doc2bow(tokenized_filtered)
    sorted_tfidf = sorted(tfidf[doc_bow], key=itemgetter(1), reverse=True)
    sorted_tfidf = pd.DataFrame([(dictionary[k],v) for k,v in sorted_tfidf],
                               columns=['word', 'score'])

    name = path.splitext(basename(fname))[0]
    tweights[name] = sorted_tfidf

val_words = set(pd.concat([v.word for v in tweights.values()]))

130it [00:00, 1610.34it/s]


In [48]:
val_dists = calc_dists(ziped_files[:1], wv)

../data//documents/0.json.gz


100%|██████████| 298/298 [01:56<00:00,  4.08it/s]
